In [ ]:
# Rule-based example using simple pattern matching
import re

text = "Please schedule a meeting with Prem tomorrow at 9 AM."

if re.search(r'\bplease\b', text.lower()) and re.search(r'\b(schedule|book|arrange)\b', text.lower()):
    print("Detected Intent: Set Meeting")
else:
    print("Intent not recognized.")


Detected Intent: Set Meeting


In [ ]:
#ML-based NLP
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from transformers import pipeline, logging
logging.set_verbosity_error()

# Load zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Input text
text = "Please schedule a meeting with John tomorrow at 10 AM."
labels = ["Set Meeting", "Cancel Meeting", "Weather Query"]

# Predict intent
result = classifier(text, labels)
print("Predicted Intent:", result['labels'][0])



config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Predicted Intent: Set Meeting


In [ ]:
from transformers import pipeline, logging
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
logging.set_verbosity_error()

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

text2 = "Will it rain tomorrow in New York?"
labels = ["Set Meeting", "Cancel Meeting", "Weather Query"]

result = classifier(text2, labels)
print("Predicted Intent:", result['labels'][0])


Predicted Intent: Weather Query


In [ ]:
import re

def detect_cancel_meeting(text):
    text = text.lower()
    if re.search(r'\b(cancel|call off|drop)\b', text) and re.search(r'\b(meeting|appointment)\b', text):
        return "Detected Intent: Cancel Meeting"
    else:
        return "Intent not recognized."

# Test case
text1 = "Can you please cancel my meeting with Alice at 2 PM?"
print(detect_cancel_meeting(text1))


Detected Intent: Cancel Meeting


In [ ]:
text3 = "I'd like to order a pizza and garlic bread for dinner."
labels = ["Set Meeting", "Cancel Meeting", "Weather Query", "Order Food"]

result = classifier(text3, labels)
print("Predicted Intent:", result['labels'][0])


Predicted Intent: Order Food


In [ ]:
def detect_set_meeting(text):
    text = text.lower()
    if re.search(r'\b(please|kindly)?\b', text) and re.search(r'\b(schedule|book|arrange|set up)\b', text) and re.search(r'\b(meeting|appointment)\b', text):
        return "Detected Intent: Set Meeting"
    else:
        return "Intent not recognized."

print("Rule-based Result:", detect_set_meeting("Please book a meeting with Sarah next Monday."))


Rule-based Result: Detected Intent: Set Meeting


In [ ]:
text4 = "Please book a meeting with Sarah next Monday."
labels = ["Set Meeting", "Cancel Meeting", "Weather Query"]

result = classifier(text4, labels)
print("ML-based Result:", result['labels'][0])


ML-based Result: Set Meeting


In [ ]:
# 1. Setup
# Install dependencies
!pip install --quiet nltk scikit-learn pandas

# Download NLTK resources
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# 2. Load Dataset
import pandas as pd

# Create SMS dataset
data = [
    ("ham", "Hey, are we still on for lunch today?"),
    ("spam", "URGENT! You've won $1000! Click here now!"),
    ("ham", "Can you pick up milk on your way home?"),
    ("spam", "FREE iPhone! Limited time offer! Call now!"),
    ("ham", "Meeting moved to 3pm tomorrow"),
    ("spam", "Congratulations! You've been selected for a special offer!"),
    ("ham", "Thanks for the birthday wishes!"),
    ("spam", "SALE ALERT: 90% off everything! Don't miss out!"),
    ("ham", "Running late, be there in 10 minutes"),
    ("spam", "You owe $500 in taxes. Pay immediately or face legal action!")
]

df = pd.DataFrame(data, columns=['label','message'])
df.head()

,label,message
0,ham,"Hey, are we still on for lunch today?"
1,spam,URGENT! You've won $1000! Click here now!
2,ham,Can you pick up milk on your way home?
3,spam,FREE iPhone! Limited time offer! Call now!
4,ham,Meeting moved to 3pm tomorrow


In [ ]:
# 3. Rule-Based Classifier
import re

# Define spam keywords
spam_keywords = {'urgent', 'free', 'offer', 'sale', 'click', 'congratulations', 'winner', 'won', 'alert'}

def rule_based_classifier(text):
    tokens = re.findall(r'\b\w+\b', text.lower())
    return 'spam' if any(word in spam_keywords for word in tokens) else 'ham'

# Apply rule-based classifier
df['pred_rule'] = df['message'].apply(rule_based_classifier)
df

,label,message,pred_rule
0,ham,"Hey, are we still on for lunch today?",ham
1,spam,URGENT! You've won $1000! Click here now!,spam
2,ham,Can you pick up milk on your way home?,ham
3,spam,FREE iPhone! Limited time offer! Call now!,spam
4,ham,Meeting moved to 3pm tomorrow,ham
5,spam,Congratulations! You've been selected for a sp...,spam
6,ham,Thanks for the birthday wishes!,ham
7,spam,SALE ALERT: 90% off everything! Don't miss out!,spam
8,ham,"Running late, be there in 10 minutes",ham
9,spam,You owe $500 in taxes. Pay immediately or face...,ham


In [ ]:
# 4. ML-Based Classifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    df['message'], df['label'], test_size=0.3, random_state=42)

# Vectorize text
vec = CountVectorizer(stop_words='english')
X_train_vec = vec.fit_transform(X_train)
X_test_vec = vec.transform(X_test)

# Train Naïve Bayes
clf = MultinomialNB().fit(X_train_vec, y_train)
df.loc[X_test.index, 'pred_ml'] = clf.predict(X_test_vec)
df

,label,message,pred_rule,pred_ml
0,ham,"Hey, are we still on for lunch today?",ham,NaN
1,spam,URGENT! You've won $1000! Click here now!,spam,ham
2,ham,Can you pick up milk on your way home?,ham,NaN
3,spam,FREE iPhone! Limited time offer! Call now!,spam,NaN
4,ham,Meeting moved to 3pm tomorrow,ham,NaN
5,spam,Congratulations! You've been selected for a sp...,spam,spam
6,ham,Thanks for the birthday wishes!,ham,NaN
7,spam,SALE ALERT: 90% off everything! Don't miss out!,spam,NaN
8,ham,"Running late, be there in 10 minutes",ham,ham
9,spam,You owe $500 in taxes. Pay immediately or face...,ham,NaN


In [ ]:
 # 5. Evaluation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    df['message'], df['label'], test_size=0.3, random_state=42)

# Vectorize text
vec = CountVectorizer(stop_words='english')
X_train_vec = vec.fit_transform(X_train)
X_test_vec = vec.transform(X_test)

# Train Naïve Bayes
clf = MultinomialNB().fit(X_train_vec, y_train)
df.loc[X_test.index, 'pred_ml'] = clf.predict(X_test_vec)
df

,label,message,pred_rule,pred_ml
0,ham,"Hey, are we still on for lunch today?",ham,NaN
1,spam,URGENT! You've won $1000! Click here now!,spam,ham
2,ham,Can you pick up milk on your way home?,ham,NaN
3,spam,FREE iPhone! Limited time offer! Call now!,spam,NaN
4,ham,Meeting moved to 3pm tomorrow,ham,NaN
5,spam,Congratulations! You've been selected for a sp...,spam,spam
6,ham,Thanks for the birthday wishes!,ham,NaN
7,spam,SALE ALERT: 90% off everything! Don't miss out!,spam,NaN
8,ham,"Running late, be there in 10 minutes",ham,ham
9,spam,You owe $500 in taxes. Pay immediately or face...,ham,NaN
